# AI Search Pull Type

# 02_Chunking Optimization
Tips for Chunking Optimization to create search indexes for RAG

## Use case
- Azure AI Search の サービス仕様ドキュメントをインプットにする。
  - https://learn.microsoft.com/ja-jp/azure/search/
- ドキュメントは OCR が必要。
- 開発者マニュアルは、構造化されたセクションとなっている。
- 各セクションは非常に詳細かつ専門性の高い技術解説が記載されており、ドキュメントサイズも大きい。
- ドキュメントには、テキスト、テーブル、図、グラフなどが含まれるが、ここでは、テキスト、テーブルデータのみを扱う。

## チャンキング設計
- Document Intelligence で、Markdown形式でテキストデータを抽出済み。
- 1つのドキュメントに大量のコンテキストが含まれており、ドキュメントサイズも大きいため、チャンキングを実施する。
- ドキュメントは、技術要素ごとに明確なセクションわけがされており、各セクションで見るとLLMが扱えないレベルのデータサイズではない。そのため、セクション単位でチャンキングする。
- 各チャンキングのContentはEmbeddingする。
- Overlapping は行わない。
- 広い意味のコンテキストを保持するために、上位2つのヘッダー（Markdown形式：#, ##）をメタデータとして保持する。例えば、検索インデックスに関する記載があった場合に、それが「キーワード検索」に属する情報なのか、「ベクトル検索」に属する情報なのかを判断するために保持する。

In [3]:
! pip install azure-search-documents==11.6.0b4
! pip install openai python-dotenv azure-identity cohere azure-ai-vision-imageanalysis
! pip install azure-storage-blob

In [2]:
import os

from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from azure.search.documents.indexes.models import (
    AIServicesVisionParameters,
    AIServicesVisionVectorizer,
    AIStudioModelCatalogName,
    AzureMachineLearningVectorizer,
    AzureOpenAIVectorizer,
    AzureOpenAIModelName,
    AzureOpenAIParameters,
    BlobIndexerDataToExtract,
    BlobIndexerParsingMode,
    CognitiveServicesAccountKey,
    DefaultCognitiveServicesAccount,
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    FieldMapping,
    HnswAlgorithmConfiguration,
    HnswParameters,
    IndexerExecutionStatus,
    IndexingParameters,
    IndexingParametersConfiguration,
    InputFieldMappingEntry,
    OutputFieldMappingEntry,
    ScalarQuantizationCompressionConfiguration,
    ScalarQuantizationParameters,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SearchIndexer,
    SearchIndexerDataContainer,
    SearchIndexerDataIdentity,
    SearchIndexerDataSourceConnection,
    SearchIndexerSkillset,
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch,
    SimpleField,
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
    VisionVectorizeSkill
)
from azure.search.documents.models import (
    HybridCountAndFacetMode,
    HybridSearch,
    SearchScoreThreshold,
    VectorizableTextQuery,
    VectorizableImageBinaryQuery,
    VectorizableImageUrlQuery,
    VectorSimilarityThreshold,
)
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
from IPython.display import Image, display, HTML
from openai import AzureOpenAI

In [5]:
# Load environment variables
load_dotenv()

# Configuration
AZURE_AI_VISION_API_KEY = os.getenv("AZURE_AI_VISION_API_KEY")
AZURE_AI_VISION_ENDPOINT = os.getenv("AZURE_AI_VISION_ENDPOINT")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
BLOB_CONNECTION_STRING = os.getenv("BLOB_CONNECTION_STRING")
BLOB_CONTAINER_NAME = os.getenv("BLOB_CONTAINER_NAME")
INDEX_NAME = "rag-search-index-pull-0621"
AZURE_SEARCH_ADMIN_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY")
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")

In [6]:
# User-specified parameter
USE_AAD_FOR_SEARCH = False  # Set this to False to use API key for authentication

def authenticate_azure_search(api_key=None, use_aad_for_search=False):
    if use_aad_for_search:
        print("Using AAD for authentication.")
        credential = DefaultAzureCredential()
    else:
        print("Using API keys for authentication.")
        if api_key is None:
            raise ValueError("API key must be provided if not using AAD for authentication.")
        credential = AzureKeyCredential(api_key)
    return credential

azure_search_credential = authenticate_azure_search(api_key=AZURE_SEARCH_ADMIN_KEY, use_aad_for_search=USE_AAD_FOR_SEARCH)

Using API keys for authentication.
